背景：



In [3]:
import os
import sys

current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from apikey import apikey, langchain_apikey

os.environ['OPENAI_API_KEY'] = apikey
os.environ['LANGCHAIN_API_KEY'] = langchain_apikey
os.environ["LANGCHAIN_TRACING_V2"] = "true"


# 1.记录到特定项目

如“概念”部分所述，LangSmith 使用项目的概念对Trace进行分组。

如果未指定，则跟踪器项目将设置为default。您可以设置 LANGCHAIN_PROJECT 环境变量，以便为整个应用程序运行配置自定义项目名称。这应该在执行程序之前完成。

In [ ]:
export LANGCHAIN_PROJECT="My Project"



# 2.在运行时更改目标项目

当全局环境变量过于宽泛时，还可以在程序运行时设置项目名称。当您想要将跟踪记录到同一应用程序中的不同项目时，这很有用。

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("user", "Question: {question}")
])

model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser



## 2-1. 直接通过langchain

In [15]:
# You can set the project name for a specific tracer instance:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")

question = "How many people live in canada as of 2023?"

chain.invoke({"question": question}, config={"callbacks": [tracer]})




'As of 2023, the estimated population of Canada is around 38 million people.'

In [17]:

# LangChain python also supports a context manager for tracing a specific block of code.
# You can set the project name using the project_name parameter.
from langchain_core.tracers.context import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    chain.invoke({"question": "How many people live in china as of 2023?"})
